In [ ]:
public enum OrderStatus
{
    Created,
    InProgress,
    Completed
}

public class Customer
{
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public string PhoneNumber { get; set; }

    public Customer(string firstName, string lastName, string phoneNumber)
    {
        FirstName = firstName;
        LastName = lastName;
        PhoneNumber = phoneNumber;
    }
}

public class Employee
{
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public string Position { get; set; }
    public int OrdersProcessed { get; set; }

    public Employee(string firstName, string lastName, string position)
    {
        FirstName = firstName;
        LastName = lastName;
        Position = position;
        OrdersProcessed = 0;
    }

    public void Notify(Order order, OrderStatus newStatus)
    {
        Console.WriteLine($"Уведомление для {FirstName} {LastName}: Заказ {order.OrderId} изменил статус на {newStatus}");
        if (newStatus == OrderStatus.Completed)
        {
            OrdersProcessed++;
        }
    }
}

public class Order
{
    public int OrderId { get; }
    public string Description { get; set; }
    public Employee AssignedEmployee { get; set; }
    public OrderStatus Status { get; private set; }
    public DateTime CreatedAt { get; }

    public Order(int orderId, string description)
    {
        OrderId = orderId;
        Description = description;
        Status = OrderStatus.Created;
        CreatedAt = DateTime.Now;
    }

    public void ChangeStatus(OrderStatus newStatus)
    {
        if (Status != newStatus)
        {
            Status = newStatus;
            OnStatusChanged(newStatus);
        }
    }

    public event Action<Order, OrderStatus> StatusChanged;

    protected virtual void OnStatusChanged(OrderStatus newStatus)
    {
        StatusChanged?.Invoke(this, newStatus);
    }
}

public class Company
{
    public string Name { get; set; }
    public List<Employee> Employees { get; set; }
    public List<Order> Orders { get; set; }

    public Company(string name)
    {
        Name = name;
        Employees = new List<Employee>();
        Orders = new List<Order>();
    }

    public void AddEmployee(Employee employee)
    {
        Employees.Add(employee);
    }

    public void AddOrder(Order order)
    {
        Orders.Add(order);
    }

    public void AssignEmployeeToOrder(int orderId, Employee employee)
    {
        var order = Orders.FirstOrDefault(o => o.OrderId == orderId);
        if (order != null)
        {
            order.AssignedEmployee = employee;
            order.StatusChanged += employee.Notify;
        }
    }

    public void ViewOrdersByStatus(OrderStatus status)
    {
        var orders = Orders.Where(o => o.Status == status).ToList();
        foreach (var order in orders)
        {
            Console.WriteLine($"Заказ {order.OrderId}: {order.Description} (Статус: {order.Status})");
        }
    }

    public void SearchOrdersByCustomer(Customer customer)
    {
        var orders = Orders.Where(o => o.Description.Contains($"Заказчик: {customer.FirstName} {customer.LastName}")).ToList();
        foreach (var order in orders)
        {
            Console.WriteLine($"Заказ {order.OrderId}: {order.Description} (Статус: {order.Status})");
        }
    }

    public void GenerateEmployeeReport()
    {
        foreach (var employee in Employees)
        {
            Console.WriteLine($"{employee.FirstName} {employee.LastName} (Должность: {employee.Position}) обработал {employee.OrdersProcessed} заказов.");
        }
    }
}


Company toyCompany = new Company("Игрушки и Ко");

Employee employee1 = new Employee("Иван", "Иванов", "Менеджер");
Employee employee2 = new Employee("Петр", "Петров", "Продавец");
toyCompany.AddEmployee(employee1);
toyCompany.AddEmployee(employee2);

Customer customer1 = new Customer("Алексей", "Смирнов", "123-456-789");
Customer customer2 = new Customer("Мария", "Иванова", "987-654-321");

Order order1 = new Order(1, $"Заказчик: {customer1.FirstName} {customer1.LastName}, Игрушки: Машинка, Кукла");
Order order2 = new Order(2, $"Заказчик: {customer2.FirstName} {customer2.LastName}, Игрушки: Робот, Конструктор");
Order order3 = new Order(3, $"Заказчик: {customer1.FirstName} {customer1.LastName}, Игрушки: Пазл, Кубики");
Order order4 = new Order(4, $"Заказчик: {customer2.FirstName} {customer2.LastName}, Игрушки: Мягкая игрушка, Кукла");
toyCompany.AddOrder(order1);
toyCompany.AddOrder(order2);
toyCompany.AddOrder(order3);
toyCompany.AddOrder(order4);

toyCompany.AssignEmployeeToOrder(1, employee1);
toyCompany.AssignEmployeeToOrder(2, employee2);
toyCompany.AssignEmployeeToOrder(3, employee1);
toyCompany.AssignEmployeeToOrder(4, employee2);

order1.ChangeStatus(OrderStatus.InProgress);
order2.ChangeStatus(OrderStatus.Completed);
order3.ChangeStatus(OrderStatus.InProgress);
order4.ChangeStatus(OrderStatus.Completed);

Console.WriteLine("\nЗаказы в работе:");
toyCompany.ViewOrdersByStatus(OrderStatus.InProgress);

Console.WriteLine("\nЗаказы завершены:");
toyCompany.ViewOrdersByStatus(OrderStatus.Completed);

Console.WriteLine("\nПоиск заказов по заказчику Алексей Смирнов:");
toyCompany.SearchOrdersByCustomer(customer1);

Console.WriteLine("\nОтчет по сотрудникам:");
toyCompany.GenerateEmployeeReport();


Уведомление для Иван Иванов: Заказ 1 изменил статус на InProgress
Уведомление для Петр Петров: Заказ 2 изменил статус на Completed
Уведомление для Иван Иванов: Заказ 3 изменил статус на InProgress
Уведомление для Петр Петров: Заказ 4 изменил статус на Completed

Заказы в работе:
Заказ 1: Заказчик: Алексей Смирнов, Игрушки: Машинка, Кукла (Статус: InProgress)
Заказ 3: Заказчик: Алексей Смирнов, Игрушки: Пазл, Кубики (Статус: InProgress)

Заказы завершены:
Заказ 2: Заказчик: Мария Иванова, Игрушки: Робот, Конструктор (Статус: Completed)
Заказ 4: Заказчик: Мария Иванова, Игрушки: Мягкая игрушка, Кукла (Статус: Completed)

Поиск заказов по заказчику Алексей Смирнов:
Заказ 1: Заказчик: Алексей Смирнов, Игрушки: Машинка, Кукла (Статус: InProgress)
Заказ 3: Заказчик: Алексей Смирнов, Игрушки: Пазл, Кубики (Статус: InProgress)

Отчет по сотрудникам:
Иван Иванов (Должность: Менеджер) обработал 0 заказов.
Петр Петров (Должность: Продавец) обработал 2 заказов.
